This is a companion notebook for the book [Deep Learning with Python, Second Edition](https://www.manning.com/books/deep-learning-with-python-second-edition?a_aid=keras&a_bid=76564dff). For readability, it only contains runnable code blocks and section titles, and omits everything else in the book: text paragraphs, figures, and pseudocode.

**If you want to be able to follow what's going on, I recommend reading the notebook side by side with your copy of the book.**

This notebook was generated for TensorFlow 2.6.

In [1]:

from tensorflow import keras

base_image_path = keras.utils.get_file(
    "sf.jpg", origin="https://img-datasets.s3.amazonaws.com/sf.jpg")
style_reference_image_path1 = keras.utils.get_file(
    "starry_night.jpg", origin="https://img-datasets.s3.amazonaws.com/starry_night.jpg")

original_width, original_height = keras.utils.load_img(base_image_path).size
img_height = 400
img_width = round(original_width * img_height / original_height)

In [2]:
import cv2
import os

def reset_folder(folder_path):
    if os.path.exists(folder_path):
        # Delete the folder and its contents
        for item in os.listdir(folder_path):
            item_path = os.path.join(folder_path, item)
            if os.path.isdir(item_path):
                # Recursively delete subfolders and files
                reset_folder(item_path)
            else:
                os.remove(item_path)
        os.rmdir(folder_path)

    # Recreate the folder
    os.makedirs(folder_path)

def video_to_frames(mp4_path, frame_folder):
    reset_folder(frame_folder)
    vidcap = cv2.VideoCapture(mp4_path)
    success, image = vidcap.read()
    count = 0

    while success:
        frame_path = os.path.join(frame_folder, f"frame{count:04d}.jpg")  # Save frame with zero-padded numbers
        cv2.imwrite(frame_path, image)
        success, image = vidcap.read()
        print('Read a new frame: ', success)
        count += 1

    # Sort the frames in the folder
    frame_files = sorted([f for f in os.listdir(frame_folder) if f.endswith('.jpg')])

    for i, frame_file in enumerate(frame_files):
        new_frame_path = os.path.join(frame_folder, f"frame{i:04d}.jpg")  # Rename the frames
        os.rename(os.path.join(frame_folder, frame_file), new_frame_path)


def frames_to_video(frame_folder, output_video_path, frame_rate):
    frame_files = [f for f in os.listdir(frame_folder) if f.endswith('.jpg')]
    frame_files.sort(key=lambda x: int(x[5:-4]))  # Sort the frames in the correct order

    frame = cv2.imread(os.path.join(frame_folder, frame_files[0]))
    height, width, layers = frame.shape

    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_video_path, fourcc, frame_rate, (width, height))

    for frame_file in frame_files:
        frame_path = os.path.join(frame_folder, frame_file)
        frame = cv2.imread(frame_path)
        out.write(frame)

    out.release()

